Урок 9. Возможна ли жизнь без PIP?
Прикрутить бота к задачам с предыдущего семинара: Создать телефонный справочник с возможностью импорта и экспорта данных в нескольких форматах. То есть данные о пользователях отправляются боту в сообщениях, а в результате сохраняются в csv.
Материалы в помощь: https://habr.com/ru/post/580408/ https://xakep.ru/2021/11/28/python-telegram-bots/#toc02.

In [ ]:
import telebot
from telebot import types
import tk

bot = telebot.TeleBot(tk.Token)

def add_to_phone_book(text, file = "phone_book.csv", splitter = ";"):
    text = text.split(" ");
    with open(file, "a") as phone_book:
        phone_book.write (splitter.join(text))
        phone_book.write ("\n")
    return "Данные добавлены"

def get_users_list(phone_book = "phone_book.csv"):
    k = ["Number", "Surname", "Name"]
    list_phone_book = []
    with open(phone_book, "r") as us:
        for i in us:
            u = i.split(";")
            user = {}
            for j in range(3):
                user[k[j]]=u[j].replace("\n","").replace("\t","")
            list_phone_book.append(user)
    return list_phone_book

def Search(phone_book, name):
    parameter = ""
    if name.isdigit():
        parameter = "Number"
    elif " " in name:
        parameter = "SN"
    else:
        parameter = "Surname"
    
    if parameter == "SN":
        return [user for user in phone_book if name in user["Surname"] + " " + user["Name"]]
    else:
        return [user for user in phone_book if name in user[parameter]]
    
    

@bot.message_handler(commands=["start"])
def start(message, res=False):
    bot.send_message(message.chat.id, 'Привет! Напиши: \n - /search, чтобы найти контакт \n - /add добавить контакт')

    
@bot.message_handler(commands=["add"])    
def start_add(message, res=False):
    bot.send_message(message.chat.id, 'Введите телефон, фамилию и имя')
@bot.message_handler(commands=["search"])    
def start_search(message, res=False):
    bot.send_message(message.chat.id, 'Введите фамилию и имя')    
    
@bot.message_handler(content_types=["text"])
def handle_text(message):
    # Если юзер нажал кнопку Добавить контакт в справочник
    if len(message.text.split()) == 2:
        user = Search(get_users_list(), message.text)[0]
        result = str(user["Surname"]) + " " + str(user["Name"]) + " ( " + str(user["Number"]) + " )"
        bot.send_message(message.chat.id, result)
    elif len(message.text.split()) == 3:
        bot.send_message(message.chat.id, add_to_phone_book(message.text))
    else:
        bot.send_message(message.chat.id, 'Некорректные данные')
    
bot.polling(none_stop=True, interval=0)